# Imports

In [67]:
import os
import numpy as np
import tensorflow as tf

from time import strftime
from PIL import Image


# Constants

In [68]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

## Getting data

In [69]:
%%time
x_train_all=np.loadtxt(X_TRAIN_PATH,delimiter=',',dtype=int)

Wall time: 42.3 s


In [70]:
%%time
y_train_all=np.loadtxt(Y_TRAIN_PATH,delimiter=',',dtype=int)

Wall time: 345 ms


In [71]:
%%time
x_test=np.loadtxt(X_TEST_PATH,delimiter=',',dtype=int)

Wall time: 7.01 s


In [72]:
%%time
y_test=np.loadtxt(Y_TEST_PATH,delimiter=',',dtype=int)

Wall time: 76 ms


# Explore

In [73]:
x_train_all.shape

(60000, 784)

In [74]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

# Data Preprocessing

In [75]:
# rescaling to reduce learning rate
x_train_all,x_test=x_train_all/255.0,x_test/255.0

In [76]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

In [77]:
values=y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [78]:
y_train_all.shape

(60000,)

In [79]:
y_train_all=np.eye(NR_CLASSES)[y_train_all]

In [80]:
y_train_all.shape

(60000, 10)

In [81]:
y_test=np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

## Creating validation dataset

In [82]:
x_val=x_train_all[:VALIDATION_SIZE]
y_val=y_train_all[:VALIDATION_SIZE]

In [83]:
x_val.shape

(10000, 784)

In [84]:
y_val.shape

(10000, 10)

In [85]:
x_train=x_train_all[VALIDATION_SIZE:]
y_train=y_train_all[VALIDATION_SIZE:]

In [86]:
x_train.shape

(50000, 784)

In [87]:
y_train.shape

(50000, 10)

# Setup tensor graph

In [88]:
tf.compat.v1.disable_eager_execution()

In [89]:

#X = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')
#Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')
X=tf.compat.v1.placeholder(tf.float32,  shape=[None, TOTAL_INPUTS], name='X')
Y = tf.compat.v1.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

## Neural Network   Architecture

### Hyper parameters

In [90]:
nr_epochs = 50
learning_rate = 1e-2

n_hidden1 = 512
n_hidden2 = 64

In [91]:
1e-2

0.01

In [92]:
# #layer 1
# with tf.compat.v1.name_scope("hidden_layer_1"):
#     initial_w1 = tf.compat.v1.truncated_normal(shape=[TOTAL_INPUTS,n_hidden1], stddev=0.1, seed=42)
#     w1 = tf.Variable(initial_value=initial_w1, name='w1')

#     initial_b1=tf.constant(value=0.0,shape=[n_hidden1])
#     b1=tf.Variable(initial_value=initial_b1, name='b1')

#     layer_in1=tf.matmul(X,w1)+b1
#     layer_out1=tf.nn.relu(layer_in1)

In [93]:
# #layer 2
# with tf.compat.v1.name_scope("hidden_layer_2"):
#     initial_w2 = tf.compat.v1.truncated_normal(shape=[n_hidden1,n_hidden2], stddev=0.1, seed=42)
#     w2 = tf.Variable(initial_value=initial_w2, name='w2')

#     initial_b2=tf.constant(value=0.0,shape=[n_hidden2])
#     b2=tf.Variable(initial_value=initial_b2, name='b1')

#     layer_in2=tf.matmul(layer_out1,w2)+b2
#     layer_out2=tf.nn.relu(layer_in2)

In [94]:
# #layer 3
# with tf.compat.v1.name_scope("output_layer"):
#     initial_w3 = tf.compat.v1.truncated_normal(shape=[n_hidden2,NR_CLASSES], stddev=0.1, seed=42)
#     w3 = tf.Variable(initial_value=initial_w3)

#     initial_b3=tf.constant(value=0.0,shape=[NR_CLASSES])
#     b3=tf.Variable(initial_value=initial_b3)

#     layer_in3=tf.matmul(layer_out2,w3)+b3
#     output=tf.nn.softmax(layer_in3)

In [95]:
# def setup_layer(input_,weight_dimension,bias_dimension,name):
#     with tf.compat.v1.name_scope(name):
#         initial_w = tf.compat.v1.truncated_normal(shape=weight_dimension, stddev=0.1, seed=42)
#         w = tf.Variable(initial_value=initial_w, name='W')

#         initial_b=tf.constant(value=0.0,shape=bias_dimension)
#         b=tf.Variable(initial_value=initial_b, name='B')

#         layer_in=tf.compat.v1.matmul(input_,w)+b
#         if(name=="output_layer"):
#             layer_out=tf.nn.softmax(layer_in)
#         else:
#             layer_out=tf.nn.relu(layer_in)
            
#         return layer_out

In [99]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        initial_w = tf.compat.v1.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.compat.v1.Variable(initial_value=initial_w, name='W')

        initial_b = tf.compat.v1.constant(value=0.0, shape=bias_dim)
        b = tf.compat.v1.Variable(initial_value=initial_b, name='B')

        layer_in = tf.compat.v1.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.compat.v1.nn.softmax(layer_in)
        else:
            layer_out = tf.compat.v1.nn.relu(layer_in)
        
        tf.compat.v1.summary.histogram('weights', w)
        tf.compat.v1.summary.histogram('biases', b)
        
        return layer_out

In [100]:
# layer_1=setup_layer(X,weight_dim=[TOTAL_INPUTS,n_hidden1],
#                     bias_dim=[n_hidden1],name="layer_1")

# layer_2=setup_layer(layer_1,weight_dim=[n_hidden1,n_hidden2],
#                     bias_dim=[n_hidden2],name="layer_2")

# output=setup_layer(layer_2,weight_dim=[n_hidden2,NR_CLASSES],
#                    bias_dim=[NR_CLASSES],name="output_layer")

In [101]:
#Adding dropout
layer_1=setup_layer(X,weight_dim=[TOTAL_INPUTS,n_hidden1],
                    bias_dim=[n_hidden1],name="layer_1")
dropout_layer=tf.compat.v1.nn.dropout(layer_1,keep_prob=0.8,name='dropout_layer')

layer_2=setup_layer(dropout_layer,weight_dim=[n_hidden1,n_hidden2],
                    bias_dim=[n_hidden2],name="layer_2")

output=setup_layer(layer_2,weight_dim=[n_hidden2,NR_CLASSES],
                   bias_dim=[NR_CLASSES],name="out")

## Tensorboard setup

In [102]:
model_name=f'{n_hidden1}-{n_hidden2}  Learning_rate {learning_rate} epochs  {nr_epochs}'

In [103]:
folder_name=f'{model_name} at {strftime("%H %M %S")}'
directory=os.path.join(LOGGING_PATH,folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print("created successfully!!!")

created successfully!!!


# Loss, Optimisation & Metrics

### Defining loss function

In [104]:
with tf.name_scope('loss_calc'):
    loss=tf.compat.v1.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits(labels=Y,logits=output))



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Defining optimizer


In [105]:
#tf.compat allows you to write code that works both in TensorFlow 1.x and 2.x.
with tf.name_scope('optimizer'):
    optimiser=tf.compat.v1.train.AdamOptimizer(learning_rate)
    train_step=optimiser.minimize(loss)

### Defining Accurracy metric


In [106]:
with tf.name_scope('accuracy_calc'):
    model_prediction=tf.compat.v1.argmax(output,axis=1)
    corrct_pred=tf.compat.v1.equal(model_prediction,tf.compat.v1.argmax(Y,axis=1))
    accurracy=tf.compat.v1.reduce_mean(tf.cast(corrct_pred,tf.float32))


In [107]:
#creating summary of accuracy ,loss  for tensorboard
with tf.name_scope('performance'):
    tf.compat.v1.summary.scalar('accuracy', accurracy)
    tf.compat.v1.summary.scalar('cost', loss)

### Visualisation in Tensorboard

In [109]:
x_image=tf.compat.v1.reshape(X,[-1,28,28,1])
tf.compat.v1.summary.image("Image_input",x_image,max_outputs=4)

<tf.Tensor 'Image_input:0' shape=() dtype=string>

## Run session

In [110]:
sess=tf.compat.v1.Session()

#### Creating file writer and merge summaries

In [111]:
merged_summary=tf.compat.v1.summary.merge_all()
train_writer=tf.compat.v1.summary.FileWriter(directory+'/train')

validation_writer=tf.compat.v1.summary.FileWriter(directory+'/validation')

In [112]:
train_writer.add_graph(sess.graph)
validation_writer.add_graph(sess.graph)



In [113]:
merged_summary

<tf.Tensor 'Merge/MergeSummary:0' shape=() dtype=string>

### initialising variables:

In [114]:
init=tf.compat.v1.global_variables_initializer()
sess.run(init)

In [115]:
#b3.eval(sess)

## Batching data

In [116]:
size_of_batch=1000

num_examples=y_train.shape[0]
nr_iterations=int(num_examples/size_of_batch)

index_in_epoch=0

In [117]:
def next_batch(batch_size,data,labels):
    global num_examples
    global index_in_epoch
    
    start=index_in_epoch
    index_in_epoch+=batch_size
    if index_in_epoch>num_examples:
        start=0
        index_in_epoch=batch_size
    end=index_in_epoch
    
    return data[start:end],labels[start:end]

## Training loop

In [118]:
##nr_epochs=4
for epoch in range(nr_epochs):
    # ============= Training Dataset =========
    for i in range(nr_iterations):
        batch_x,batch_y=next_batch(size_of_batch,data=x_train,labels=y_train)
        feed_dictionary={X:batch_x,Y:batch_y}
        sess.run(train_step,feed_dict=feed_dictionary)
        #tensorboard remebers things of session using summary
    batch_accuracy,s=sess.run(fetches=[accurracy,merged_summary],feed_dict=feed_dictionary)
    train_writer.add_summary(s,epoch)
    print(f"Epoch {epoch} | Training accuracy {batch_accuracy}")
    
    # ================== Validation ======================
    feed_dictionary={X:x_val,Y:y_val}
    summary=sess.run(fetches=merged_summary,feed_dict=feed_dictionary)
    validation_writer.add_summary(summary,epoch)
print(" Job done !!!")

Epoch 0 | Training accuracy 0.8560000061988831
Epoch 1 | Training accuracy 0.9710000157356262
Epoch 2 | Training accuracy 0.9739999771118164
Epoch 3 | Training accuracy 0.9819999933242798
Epoch 4 | Training accuracy 0.9750000238418579
Epoch 5 | Training accuracy 0.984000027179718
Epoch 6 | Training accuracy 0.9810000061988831
Epoch 7 | Training accuracy 0.9869999885559082
Epoch 8 | Training accuracy 0.9760000109672546
Epoch 9 | Training accuracy 0.9760000109672546
Epoch 10 | Training accuracy 0.9750000238418579
Epoch 11 | Training accuracy 0.9779999852180481
Epoch 12 | Training accuracy 0.9710000157356262
Epoch 13 | Training accuracy 0.9789999723434448
Epoch 14 | Training accuracy 0.984000027179718
Epoch 15 | Training accuracy 0.9810000061988831
Epoch 16 | Training accuracy 0.9860000014305115
Epoch 17 | Training accuracy 0.9789999723434448
Epoch 18 | Training accuracy 0.9789999723434448
Epoch 19 | Training accuracy 0.9779999852180481
Epoch 20 | Training accuracy 0.9760000109672546
Epoc

## Make prediction

In [ ]:
imag=Image.open("MNIST/test_img.png")
imag

In [ ]:
bw=imag.convert('L')#converting to grayscale

In [ ]:
img_array = np.invert(bw)## white to black and black to white

In [ ]:
img_array.shape

In [ ]:
"""The numpy.ravel() functions returns contiguous 
flattened array(1D array with all the input-array elements and with the same type as it"""
test_img = img_array.ravel()

In [ ]:
test_img.shape

In [ ]:
prediction=sess.run(fetches=tf.argmax(output,axis=1),feed_dict={X:[test_img]})

In [ ]:
prediction

# Testing and Evaluation

In [ ]:
test_accuracy=sess.run(fetches=accurracy,feed_dict={X:x_test,Y:y_test})

In [ ]:
test_accuracy

## Saving the model

In [ ]:
outputs_={"accuracy_calc/prediction":model_prediction}
inputs__={"X":X}
path="Saved_models"

tf.compat.v1.saved_model.simple_save(session=sess,export_dir=path,inputs=inputs__,outputs=outputs_)

## Resetting for next run

In [ ]:
# train_writer.close()
# validation_writer.close()
# sess.close()
# tf.compat.v1.reset_default_graph()